# 🚕 Chicago Taxi Demand & Weather Impact Analysis

## Context
As an analyst in Zuber, a transportation company in Chicago, need to understand:
- Passenger preferences
- The most active companies
- Neighborhoods with the highest demand
- The impact of weather on trip duration

## Objectives
1. Analyze trip volume by company
2. Identify the most popular destinations
3. Evaluate the impact of weather on trip duration
4. Test statistical hypotheses

## 📊 SQL Analysis

Queries performed:
- Trips by company
- Yellow/Blue companies
- Top companies vs. Others
- Loop → O'Hare trips
- Integration with weather data


### Query 1 – Trips by company

```sql
SELECT
    c.company_name,
    COUNT(t.trip_id) AS trips_amount
FROM trips t
INNER JOIN cabs c ON c.cab_id = t.cab_id
WHERE CAST(start_ts AS date) IN ('2017-11-15', '2017-11-16')
GROUP BY c.company_name
ORDER BY trips_amount DESC;


### Query 2 – Trips by company (Yellow/Blue) on November 7th, 2017

```sql
SELECT
    c.company_name,
    COUNT(t.trip_id) AS trips_amount
FROM trips t
INNER JOIN cabs c ON c.cab_id = t.cab_id
WHERE
    (c.company_name LIKE '%Yellow%' OR c.company_name LIKE '%Blue%')
    AND CAST(start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY c.company_name;

---

### Query 3 – Top Companies vs Others

```sql
SELECT
    CASE WHEN c.company_name IN ('Flash Cab', 'Taxi Affiliation Services') THEN c.company_name
        ELSE 'Other'
        END AS company,
    COUNT(t.trip_id) AS trips_amount
FROM trips t
INNER JOIN cabs c ON c.cab_id = t.cab_id
WHERE
    CAST(t.start_ts AS date) >= '2017-11-01'
    AND CAST(t.start_ts AS date) < '2017-11-08'
GROUP BY company
ORDER BY trips_amount DESC;
---

### Query 4 – Loop → O'Hare trips

```sql
SELECT
    neighborhood_id,
    name
FROM neighborhoods
WHERE name IN ('O''Hare', 'Loop');
---

### Query 5 – Integration with weather data

```sql
SELECT
    t.start_ts, --Divide the hours into groups
    CASE WHEN w.description LIKE '%rain%' THEN 'Bad'
        WHEN w.description LIKE '%storm%' THEN 'Bad'
        ELSE 'Good'
        END AS weather_conditions,
    t.duration_seconds	- Duration of each trip.
FROM trips t
INNER JOIN weather_records w ON w.ts = t.start_ts
WHERE
    w.description IS NOT NULL	     - Ignore trips for which no data
    AND t.pickup_location_id = 50	     - Trips that begin in the Loop
    AND t.dropoff_location_id = 63	     –-Trips that end at OHare
    AND EXTRACT(DOW FROM t.start_ts) = 6	     - Travel on Saturdays
ORDER BY t.trip_id;
---

##Python

**project_sql_result_01.csv***: It contains company_name and trips_amount (number of trips for each taxi company on November 15 and 16, 2017).

**project_sql_result_04.csv**: It contains dropoff_location_name (Chicago neighborhoods where the trips ended) and average_trips (the average number of trips that ended in each neighborhood in November 2017).

In [5]:
#Import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#Load dataset into a DataFrame
company_trips = pd.read_csv('/datasets/project_sql_result_01.csv')
destination = pd.read_csv('/datasets/project_sql_result_04.csv')

In [ ]:
#Trips by company (November 15th and 16th)
# Create a list of the top 10 companies based on the number of trips
top10_companies = company_trips.groupby('company_name')['trips_amount'].sum().sort_values(ascending=False).head(10)
print(top10_companies)

#Visual Distribution - Bar Chart
top10_companies.plot(kind='barh', color='gold')
plt.title('Trips by Company (November 15th and 16th, 2017')
plt.xlabel('Company Name')
plt.ylabel('Number of trips')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#Identify the top 10 Destination Neighborhoods in November 2017
#Create a list with the top 10 Destination Neighborhoods
top10_destinations = destination.groupby('dropoff_location_name')['average_trips'].sum().sort_values(ascending=False).head(10)
print(top10_destinations)

#Visual Distribution - Bar Chart
top10_destinations.plot(kind='bar', rot=60, color='teal')
plt.title('Average Trips by Neighborhood Destination in November 2017')
plt.xlabel('Neighborhood')
plt.ylabel('Average trips')
plt.show()

**project_sql_result_07.csv**: It contains data on trips from the Loop to O'Hare International Airport.
- start_ts: date and time of pickup
- weather_conditions: weather conditions at the time the trip began
- duration_seconds: duration of the trip in seconds

**Test the hypothesis**

*"Average trip length from the Loop to O'Hare International Airport changes on rainy Saturdays."*

***Null Hypothesis (H₀)***: The average duration of trips from the Loop to O'Hare International Airport on rainy Saturdays is equal to the average duration on non-rainy Saturdays.

***Alternative Hypothesis (H₁)***: The average duration of trips from the Loop to O'Hare International Airport on rainy Saturdays is different from the average duration on non-rainy Saturdays.

**Explanation**:
This is a two-tailed test, because we are testing whether rainy Saturdays affect the average trip duration, without assuming whether it increases or decreases.
The null hypothesis (H₀) assumes no difference between rainy and non-rainy Saturdays.
The alternative hypothesis (H₁) considers the possibility of a significant difference in average trip duration.

In [ ]:
#Load dataset into a DataFrame
saturday_trips = pd.read_csv('/datasets/project_sql_result_07.csv')

#Create a new column with the duration in minutes
saturday_trips['duration_minutes'] = saturday_trips['duration_seconds'] / 60

#Verify
saturday_trips.head()

In [ ]:
#Import library for hypothesis testing
from scipy import stats as st

#Extract duration for "Good" and "Bad" weather conditions
good = saturday_trips[saturday_trips['weather_conditions'] == ['Good']['duration_minutes']
bad = saturday_trips[saturday_trips['weather_conditions'] == ['Bad']['duration_minutes']

#Significance value
alpha= 0.05

#Two-sample t-test for unequal variances (Welch's t-test)
duration_mean_results = st.ttest_ind(good, bad, equal_var=False)
print("Pvalue:", duration_mean_results.pvalue)

if duration_mean_results.pvalue < alpha:
    print("We reject the null hypothesis: the average duration of trips are significantly different.")
else:
    print("We cannot reject the null hypothesis: there is no evidence of a significant difference.")

In [ ]:
print("Good weather:")
print("Mean:", good.mean())
print("Std:", good.std())
print("Count:", good.count(), "\n")

print("Bad weather:")
print("Mean:", bad.mean())
print("Std:", bad.std())
print("Count:", bad.count())

In [ ]:
#Visualization Boxplot
plt.boxplot([good, bad], labels=['Good', 'Bad'])
plt.ylabel('Trip Duration (minutes)')
plt.title('Trip Duration by Weather Condition on Saturdays')
plt.show()

In [ ]:
#Visualization Histogram
plt.hist(good, bins=20, alpha=0.6, label='Good weather', color='skyblue')
plt.hist(bad, bins=20, alpha=0.6, label='Bad weather', color='salmon')
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Number of Trips')
plt.title('Distribution of Trip Duration on Saturdays')
plt.legend()
plt.show()

## ✅Conclusions
- Flash Cab dominates the market
- ​​Loop is the main destination
- Rain increases the average trip duration

## 📉 Business Impact
These findings allow for:
- Improved dynamic pricing
- Adjusted time estimates
- Optimized fleet allocation
- Prediction of weather-related delays